## Summary

Resource: https://scikit-learn.org/stable/developers/develop.html#rolling-your-own-estimator

Resumen del capítulo que explica cómo desarrollar objetos que interactúan con scikit-learn pipelines y model selection tools.

## APIs of scikit-learn objects

Tenemos dos tipos de estimadores, los **simples** (la mayoría), como la regresión logística (`LogisticRegression`) o un RF (`RandomForestClassifier`). Luego tenemos los **meta-estimators** que son *wraps* de otros estimadores (ejemplos son `Pipeline`, o `GridSearchCV`). 

### Instancia

Al crear la instancia del objeto tenemos que permitir aceptar como argumentos a constantes que determinen el comportamiento del estimador. Pero no debemos dejar que la *training data* sea un argumento, como es el caso del método `fit`.

Idealmente, además, los argumentos del `__init__` tienen que ser keyword arguments con default value. Es decir, el usuario debería poder instanciar el estimador sin pasar argumentos. Solo en algunos casos donde es muy raro setear el default, se deja sin. Además, notar que no se tienen que documentar en la sección de "Atributos" sino en la de "Parámetros". 

Además, cada keyword argument aceptado en el constructor tiene que corresponderse con un atributo de la instancia. Scikit-learn depende de esto para encontrar atributos relevantes a setear en un estimador cuando se implementa model selection. Además, no se tienen que modificar los parámetros dentro del init, lo que implica que idealmente no deben ser mutable objects (como listas o diccionarios). 

En el __init__ no tiene que incluirse input validation y tampoco se tienen que settear los parámetros con trailing "_".

### Different objects

Los principales objetos de scikit-learn son:

1. **Estimator**

Es el objeto base, implementa un fit method para aprender de los datos. Ejemplo:

> estimator = estimator.fit(data) ó estimator.fit(X, y)

fit es usualmente el método que se implementa "a mano", los otros (como set_params y get_params) se implementan en BaseEstimator y se deben heredar. El fit method toma la training data como argumentos, aunque también se suele sumar otra metadata como "sample_weight".

Note that the model is fitted using X and y, but the object holds no reference to X and y. There are, however, some exceptions to this, as in the case of precomputed kernels where this data must be stored for use by the predict method.

- X tiene que ser un array-like de shape (n_samples, n_features)
- y es un array-like de shape (n_samples,)
- kwargs optional data-dependent parameters

El numero de samples (X.shape[0], y.shape[0]) tiene que ser el mismo, y si el requerimiento no se cumple, debe raise ValueError. 

y might be ignored in the case of unsupervised learning. However, to make it possible to use the estimator as part of a pipeline that can mix both supervised and unsupervised transformers, even unsupervised estimators need to accept a y=None keyword argument in the second position that is just ignored by the estimator. For the same reason, fit_predict, fit_transform, score and partial_fit methods need to accept a y argument in the second place if they are implemented.

The method should return the object (self). This pattern is useful to be able to implement quick one liners in an IPython session such as:

> y_predicted = SGDClassifier(alpha=10).fit(X_train, y_train).predict(X_test)

Cualquier parámetro que tiene valor asignado antes de tener acceso a los datos debe ser un __init__ keyword argument. Idealmente, los parametros de fit deben estar restringidos a data dependent variables. Una Gram matrix y una matriz de afinidad, que se precomputan desde la matriz de datos X, son data dependent. A tolerance stopping criterion **tol** no es directamente data dependent (aunque el valor óptimo puede venir asociado a una función score que sí es data dependent).

Adicionalmente, cuando se llama al método fit, toda llamada previa del mismo método debe ser ignorada (si llamé estimator.fit(X1) y luego a estimator.fit(X2), debe ser lo mismo que solo llamar a estimator.fit(X2)). Excepciones a la regla son cuando fit depende de cierto proceso random o cuando el hiperparámetro **warm_start** es True para estimadores que lo soportan. **warm_start=True** significa que el estado previo de los parámetros de entrenamiento del estimador se reusan en lugar de usar la inicialización por default. 

#### Estimated Attributes

Según la convención de scikit, los atributos que se quieren exponer públicamente a los usuarios, que han sido estimados o aprendidos de los datos, deben siempre tener un nombre que termina con trailing underscore, por ejemplo los coeficientes de algún estimador de regresión se deben almacenar como **coef_** atributo luego de que se llamó fit. Y todos los atributos que se aprenden en el proceso que se quieren almacenar pero no exponer al usuario, deben tener un leading underscore (como "_intermediate_coefs"). Se tienen que documentar los del primer grupo como "Atributos" pero no hay necesidad de documentar el segundo. 

Se espera que se sobreescriban los atributos estimados cuando se llama a fit una segunda vez. 

#### Universal Attributes

Los estimadores que esperan un input tabular tienen que setear un atributo llamado **n_features_in_** en fit para indicar el numero de features que el estimador tienen que esperar en los próximos llamados a predict o transform.

Si a los estimadores se les da un dataframe, deben setear un **features_names_in_** para indicar los features names de la input data. Usar **validate_data** para setear esos atributos automaticamente.

2. **Predictor**

Para supervised o unsupervised learning. Implementamos con:

> prediction = predictor.predict(data)

Cuando usamos algoritmos de clasificación también solemos tener una forma de cuantificar la certidumbre de la predicción, usando decision_function o predict_proba. 

3. **Transformer** 

Para modificar datos de forma supervisada o no supervisada. Por ejemplo, añadiendo, cambiando o removiendo columnas (pero no así filas). Se implementa como:

> new_data = transformer.transform(data)

Cuando fittear y transformar se puede performar de forma más eficiente en conjunto implementamos: 

> new_data = transformer.fit_transform(data)

4. **Model** 

Un modelo puede devolver medidas de *goodness of fit* o *likelihood of unseen data*, se implementa como (mayor es mejor):

> score = model.score(data)


### Rolling your own estimator

Se puede chequear si el estimador se adhiere a la interfaz de scikit-learn y a sus estándares corriendo **check_estimator** en una instancia. Tambien se puede usar el decorator pytest **parametrize_with_checks** 

> from sklearn.utils.estimator_checks import check_estimator
> from sklearn.tree import DecisionTreeClassifier
> check_estimator(DecisionTreeClassifier())  # passes

Hay dos formas de lograr la interfaz correcta:

1. Project template
Recurso: https://github.com/scikit-learn-contrib/project-template/

2. base.BaseEstimator y mixins

Solemos usar duck typing en lugar de checkear con isinstance, lo que implica que es tecnicamente posible implementar un estimador sin heredar de scikit classes. However, if you don’t inherit from the right mixins, either there will be a large amount of boilerplate code for you to implement and keep in sync with scikit-learn development, or your estimator might not function the same way as a scikit-learn estimator.

Notar que los mixins tienen que estar a la izquierda de BaseEstimator para un propio MRO (Method Resolution Order, se rige por el algoritmo C3 que define en qué orden se priorizan métodos o atributos en una herencia múltiple)

### get_params and set_params

Todos los estimadores en scikit tienen las funciones:
1. get_params = no tiene argumentos y devuelve un diccionario de los __init__ parameters con sus valores. Tiene un keyword argument **deep** que recibe un boolean para determinar si el método deberia devolver los parámetros de sub-estimadores (solo relevantes para meta-estimators). Los nombres de los sub-estimators tienen nombres que se definen en el Pipeline object. 
2. set_params: toma los parámetros del __init__ como keyword y los devuelve en un diccionario y setea los parametros usando ese diccionario. Te devuelve el estimador. Se suele usar en contexto de grid search. 

### Cloning

base.clone, FrozenEstimator

## Estimator types

Estimadores simples
1. **Transformers** heredan de TransformerMixin e implementan un **transform** method. Deben tomar el input y transformarlo de algún modo. NUNCA deben cambiar el número de input samples

2. **Regressors** heredan de RegressorMixin e implementan un **predict** method. They should accept numerical y in their fit method. Regressors use r2_score by default in their score method.

3. **Classifiers** heredan de ClassifierMixin.  If it applies, classifiers can implement decision_function to return raw decision values, based on which predict can make its decision. If calculating probabilities is supported, classifiers can also implement predict_proba and predict_log_proba.

Classifiers should accept y (target) arguments to fit that are sequences (lists, arrays) of either strings or integers. They should not assume that the class labels are a contiguous range of integers; instead, they should store a list of classes in a classes_ attribute or property. The order of class labels in this attribute should match the order in which predict_proba, predict_log_proba and decision_function return their values. The easiest way to achieve this is to put:

> self.classes_, y = np.unique(y, return_inverse=True)

in fit. This returns a new y that contains class indexes, rather than labels, in the range [0, n_classes).

A classifier’s predict method should return arrays containing class labels from classes_. In a classifier that implements decision_function, this can be achieved with:

> def predict(self, X):
>    D = self.decision_function(X)
>    return self.classes_[np.argmax(D, axis=1)]

The multiclass module contains useful functions for working with multiclass and multilabel problems.

4. **Clustering** heredan de ClusterMixin. Aceptan un y parameter en su fit method, pero debe ser ignorado. Clustering algorithms should set a labels_ attribute, storing the labels assigned to each sample. If applicable, they can also implement a predict method, returning the labels assigned to newly given samples.

### Tags

### Developer API for set_output

### Developer API for check_is_fitted

Por default check_is_fitted mira si hay atributos con un trailing underscore. An estimator can change the behavior by implementing a __sklearn_is_fitted__ method taking no input and returning a boolean.

### Developer API for HTML representation

(está en fase experimental). 

The raw HTML representation is obtained by invoking the function **estimator_html_repr** on an estimator instance.

To customize the URL linking to an estimator’s documentation (i.e. when clicking on the “?” icon), override the **_doc_link_module** and **_doc_link_template** attributes. In addition, you can provide a **_doc_link_url_param_generator** method. Set **_doc_link_module** to the name of the (top level) module that contains your estimator. If the value does not match the top level module name, the HTML representation will not contain a link to the documentation. For scikit-learn estimators this is set to "sklearn".

The **_doc_link_template** is used to construct the final URL. By default, it can contain two variables: estimator_module (the full name of the module containing the estimator) and estimator_name (the class name of the estimator). If you need more variables you should implement the **_doc_link_url_param_generator** method which should return a dictionary of the variables and their values. This dictionary will be used to render the **_doc_link_template**.

### Coding guidelines

1. Seguir de cerca los guidelines en PEP8
Adicionan:
2. Usar underscores para separar palabras en nombres de cosas que no son clases (n_samples en lugar de nsamples).
3. Evitar multiple statements en una linea. Preferir una linea de return luego de una sentencia de control de flujo (if/for)
4. Use relative imports for references inside scikit-learn.
5. Don't use import * in any case. 
6. Use the numpy docstring standard in all your docstrings.

### Input validation

The module sklearn.utils contains various functions for doing input validation and conversion. Sometimes, np.asarray suffices for validation; do not use np.asanyarray or np.atleast_2d, since those let NumPy’s np.matrix through, which has a different API (e.g., * means dot product on np.matrix, but Hadamard product on np.ndarray).

In other cases, be sure to call check_array on any array-like argument passed to a scikit-learn API function. The exact parameters to use depends mainly on whether and which scipy.sparse matrices must be accepted.

For more information, refer to the Utilities for Developers page.

### Random Numbers

If your code depends on a random number generator, do not use numpy.random.random() or similar routines. To ensure repeatability in error checking, the routine should accept a keyword random_state and use this to construct a numpy.random.RandomState object. See sklearn.utils.check_random_state in Utilities for Developers.

If you use randomness in an estimator instead of a freestanding function, some additional guidelines apply.

First off, the estimator should take a random_state argument to its __init__ with a default value of None. It should store that argument’s value, unmodified, in an attribute random_state. fit can call check_random_state on that attribute to get an actual random number generator. If, for some reason, randomness is needed after fit, the RNG should be stored in an attribute random_state_. 

The reason for this setup is reproducibility: when an estimator is fit twice to the same data, it should produce an identical model both times, hence the validation in fit, not __init__.

### Numerical assertion in tests
When asserting the quasi-equality of arrays of continuous values, do use sklearn.utils._testing.assert_allclose.